In [1265]:
import pandas as pd
from datetime import datetime
import numpy as np

In [1266]:
sites_file_path = './data/sites_geocoded_04182024.csv'
sites_df = pd.read_csv(sites_file_path)

In [1267]:
sites_df['site_name'] = sites_df['site_name'].str.slice(0, 50)

In [1268]:
dem_path = './data/combined_data_04192024.csv'
dem_df = pd.read_csv(dem_path)

In [1269]:
import re

In [1270]:
# Function to standardis site names for both demographic and EV plan files
def standardize_name(name):
    return re.sub(r'\W+', '', name.lower())

sites_df['standardized_site_name'] = sites_df['site_name'].apply(standardize_name)
dem_df['standardized_site_name'] = dem_df['site_name'].apply(standardize_name)

In [1271]:
# Rename county_desc column in demographic file
dem_df.rename(columns={'county_desc': 'County'}, inplace=True)

In [1272]:
# Fill all NAs in demographic files
dem_df.fillna(0, inplace=True)

In [1273]:
# Merge the files based on the County, standardized_site_name, and election fields. Uses outer join to maintain all rows from both dataframes
full_df = pd.merge(dem_df,sites_df, left_on=['County', 'standardized_site_name', 'election'], right_on=['County','standardized_site_name', 'election'], how='outer')

In [1274]:
# Create the Discrepancies column to label sites that aren't consistent between the two files
full_df['Discrepancies'] = np.where(full_df['site_name_x'].isna(), 'In EV plan, no voters',
                            np.where(full_df['site_name_y'].isna(), 'Had voters, not in EV plan', 'In EV plan, had voters'))

In [1275]:
# Fill in site_name on rows where the wasn't an demographic file name for it, but there was a EV plan file name for it
full_df['site_name_x'] = np.where(full_df['site_name_x'].isna(), full_df['site_name_y'], full_df['site_name_x'])

In [1276]:
full_df.to_csv("full_df.csv", index=False)

In [1277]:
# Get a list of columns in the full_df dataframe
column_list = full_df.columns.tolist()

In [1278]:
# List of columns to keep moving forward
columns_to_keep = ['County', 'election','site_name_x', 'address','Date', 'Time','main_site',  'site_total','n','sdr_percent','age_percent_Age 18 - 25', 'age_percent_Age 26 - 40', 'age_percent_Age 41 - 65', 'age_percent_Age Over 66', 'age_percent_NA', 'n_Age 18 - 25', 'n_Age 26 - 40', 'n_Age 41 - 65', 'n_Age Over 66', 'n_NA','ethnicity_percent_HISPANIC or LATINO', 'ethnicity_percent_NOT HISPANIC or NOT LATINO', 'ethnicity_percent_UNDESIGNATED ETHNICITY', 'n_HISPANIC or LATINO', 'n_NOT HISPANIC or NOT LATINO', 'n_UNDESIGNATED ETHNICITY', 'race_percent_ASIAN', 'race_percent_BLACK or AFRICAN AMERICAN', 'race_percent_INDIAN AMERICAN or ALASKA NATIVE', 'race_percent_OTHER', 'race_percent_TWO or MORE RACES', 'race_percent_UNDESIGNATED RACE', 'race_percent_WHITE', 'n_ASIAN', 'n_BLACK or AFRICAN AMERICAN', 'n_INDIAN AMERICAN or ALASKA NATIVE', 'n_OTHER', 'n_TWO or MORE RACES', 'n_UNDESIGNATED RACE', 'n_WHITE', 'race_percent_NATIVE HAWAIIAN or PACIFIC ISLANDER', 'n_NATIVE HAWAIIAN or PACIFIC ISLANDER', 'Latitude', 'Longitude', 'Discrepancies']

In [1279]:
# New dataframe with only columns to keep
final_df = full_df[columns_to_keep]

In [1280]:
from datetime import datetime

In [1281]:
# function to calculate number of hours each site was open

def calculate_hours(time_range):
    # Check if time_range is a string
    if not isinstance(time_range, str):
        return 0  # Return 0 for non-string inputs, including NaN

    try:
        # Replace 'a.m.' with 'AM' and 'p.m.' with 'PM'
        time_range = time_range.replace('a.m.', 'AM').replace('p.m.', 'PM')

        # Split the time range into start and end times
        start_time_str, end_time_str = time_range.split(' - ')

        # Define the time format
        time_format = '%I:%M %p'

        # Convert start and end times to datetime objects
        start_time = datetime.strptime(start_time_str, time_format)
        end_time = datetime.strptime(end_time_str, time_format)

        # Calculate the duration in hours
        duration = (end_time - start_time).seconds / 3600  # Convert seconds to hours

        return duration

    except Exception as e:
        print(f"Error processing time range '{time_range}': {e}")
        return 0  # Also return 0 in case of any other error during processing

In [1282]:
# Applying the function
final_df.loc[:,'hours_open'] = final_df.loc[:,'Time'].apply(calculate_hours)

/var/folders/nq/tndcfcv94ld_dtqs_sgc70000000gq/T/ipykernel_2368/3475604174.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.loc[:,'hours_open'] = final_df.loc[:,'Time'].apply(calculate_hours)


In [1283]:
# Converting Date column to Date formate so that it can be used to divide up the days for analysis
final_df.loc[:, 'Date'] = pd.to_datetime(final_df.loc[:, 'Date'])

In [1284]:
# Label each day with day of week
def label_day_of_week(day):
    if day == 5:
        return "Saturday"
    elif day == 6:
        return "Sunday"
    else:
        return "Weekday"

In [1285]:
final_df['Date'] = pd.to_datetime(final_df['Date'])

/var/folders/nq/tndcfcv94ld_dtqs_sgc70000000gq/T/ipykernel_2368/1069453746.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Date'] = pd.to_datetime(final_df['Date'])


In [1286]:
final_df.loc[:, 'Day Type'] = final_df.loc[:,'Date'].dt.dayofweek.apply(label_day_of_week)

/var/folders/nq/tndcfcv94ld_dtqs_sgc70000000gq/T/ipykernel_2368/4268023474.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.loc[:, 'Day Type'] = final_df.loc[:,'Date'].dt.dayofweek.apply(label_day_of_week)


In [1287]:
# Counting the number of each type of day based on these columns unique sites and election
day_counts = final_df.groupby(['County', 'site_name_x','address', 'election', 'Day Type']).size().unstack(fill_value=0)

In [1288]:
# Renaming the resulting columns
day_counts.rename(columns={'Weekday': 'Weekday Count', 'Saturday': 'Saturday Count', 'Sunday': 'Sunday Count'}, inplace=True)

In [1289]:
# Add Saturday and Sunday counts to get a weekend count
day_counts['Weekend Count'] = day_counts['Saturday Count'] + day_counts['Sunday Count']

In [1290]:
# Add weekend and weekday count to get all day count
day_counts['Total Days Count'] = day_counts['Weekend Count'] + day_counts['Weekday Count']

In [1291]:
# Need to remember why I added this step
day_counts.reset_index(inplace=True)

In [1292]:
day_counts.head()

Day Type,County,site_name_x,address,election,Saturday Count,Sunday Count,Weekday Count,Weekend Count,Total Days Count
0,ALAMANCE,ALAMANCE COUNTY BOARD OF ELECTIONS OFFICE,"115 S MAPLE ST, GRAHAM, NC 27253",2017-10,1,0,12,1,13
1,ALAMANCE,ALAMANCE COUNTY BOARD OF ELECTIONS OFFICE,"115 S MAPLE ST, GRAHAM, NC 27253",2017-11,1,0,12,1,13
2,ALAMANCE,ALAMANCE COUNTY BOARD OF ELECTIONS OFFICE,"115 S MAPLE ST, GRAHAM, NC 27253",2019-10,1,0,13,1,14
3,ALAMANCE,ALAMANCE COUNTY BOARD OF ELECTIONS OFFICE,"115 S MAPLE ST, GRAHAM, NC 27253",2019-11,1,0,13,1,14
4,ALAMANCE,ALAMANCE COUNTY OFFICE ANNEX BUILDING (AUDITOR...,"201 W ELM ST, GRAHAM, NC 27253",2018-11,2,0,13,2,15


In [1293]:
# Convert hours open to numeric
final_df.loc[:,'hours_open'] = pd.to_numeric(final_df['hours_open'], errors='coerce')

In [1294]:
# For each site/election combo, sum the total hours open
hours_open_sum = final_df.groupby(['County', 'address', 'election'])['hours_open'].sum().reset_index(name='Total Hours Open')

In [1295]:
stats_df = pd.merge(day_counts.reset_index(), hours_open_sum, on=['County', 'address', 'election'], how='left')

In [1296]:
sites_dem_df = final_df.drop(columns=['Date', 'Time', 'hours_open', 'Day Type'])

In [1297]:
sites_dem_df = sites_dem_df.drop_duplicates()

In [1252]:
merged_df = pd.merge(sites_dem_df,stats_df, on=['County', 'site_name_x', 'address', 'election'], how='outer')

In [1253]:
unique_times = final_df.groupby(['County','address', 'election', 'Day Type'])['Time'].unique().unstack()

In [1254]:
unique_times = unique_times.reset_index().fillna('')

In [1255]:
final_df_with_times = pd.merge(merged_df, unique_times, on=['County', 'address', 'election'], how='outer')

In [1256]:
other_race_columns_n = ['n_INDIAN AMERICAN or ALASKA NATIVE', 'n_OTHER', 'n_TWO or MORE RACES', 'n_UNDESIGNATED RACE', 'n_NATIVE HAWAIIAN or PACIFIC ISLANDER']
other_race_columns_percent = ['race_percent_INDIAN AMERICAN or ALASKA NATIVE', 'race_percent_OTHER', 'race_percent_TWO or MORE RACES', 'race_percent_UNDESIGNATED RACE', 'race_percent_NATIVE HAWAIIAN or PACIFIC ISLANDER']
final_df_with_times['n_Other Race']= final_df_with_times[other_race_columns_n].sum(axis=1)
final_df_with_times['percent_Other Race']= final_df_with_times[other_race_columns_percent].sum(axis=1)

In [1257]:
final_df_with_times['election_date'] = final_df_with_times['election'].str.replace("-", "", regex=False)

In [1258]:
final_df_with_times.drop('index', axis=1, inplace=True)

In [1259]:
final_df_with_times.iloc[:, 6:40] = final_df_with_times.iloc[:, 6:40].fillna(0)
final_df_with_times.iloc[:, 43:49] = final_df_with_times.iloc[:, 43:49].fillna(0)

In [1260]:
rename_dict = {
    'County': 'County',
    'election': 'Election',
    'site_name_x': 'Site_Name',
    'address': 'Address',
    'main_site': 'CBOE_or_Alternate',
    'site_total': 'Site_Total',
    'n': 'Number_of_SDR_Voters',
    'sdr_percent': 'Percent_SDR_Voters',
    'age_percent_Age 18 - 25': 'Percent_of_Voters_Age_18-25',
    'age_percent_Age 26 - 40': 'Percent_of_Voters_Age_26-40',
    'age_percent_Age 41 - 65': 'Percent_of_Voters_Age_41-65',
    'age_percent_Age Over 66': 'Percent_of_Voters_Age_Over_66',
    'age_percent_NA': 'Percent_of_Voters_Age_NA',
    'n_Age 18 - 25': 'Number_of_Voters_Age_18-25',
    'n_Age 26 - 40': 'Number_of_Voters_Age_26-40',
    'n_Age 41 - 65': 'Number_of_Voters_Age_41-65',
    'n_Age Over 66': 'Number_of_Voters_Age_Over_66',
    'n_NA': 'Number_of_Voters_Age_NA',
    'ethnicity_percent_HISPANIC or LATINO': 'Percent_of_Voters_HISPANIC_or_LATINO',
    'ethnicity_percent_NOT HISPANIC or NOT LATINO': 'Percent_of_Voters_NOT_HISPANIC_or_NOT_LATINO',
    'ethnicity_percent_UNDESIGNATED ETHNICITY': 'Percent_of_Voters_UNDESIGNATED_ETHNICITY',
    'n_HISPANIC or LATINO': 'Number_of_Voters_HISPANIC_or_LATINO',
    'n_NOT HISPANIC or NOT LATINO': 'Number_of_Voters_NOT_HISPANIC_or_NOT_LATINO',
    'n_UNDESIGNATED ETHNICITY': 'Number_of_Voters_UNDESIGNATED_ETHNICITY',
    'race_percent_ASIAN': 'Percent_of_Voters_ASIAN',
    'race_percent_BLACK or AFRICAN AMERICAN': 'Percent_of_Voters_BLACK_or_AFRICAN_AMERICAN',
    'race_percent_INDIAN AMERICAN or ALASKA NATIVE': 'Percent_of_Voters_INDIAN_AMERICAN_or_ALASKA_NATIVE',
    'race_percent_OTHER': 'Percent_of_Voters_OTHER',
    'race_percent_TWO or MORE RACES': 'Percent_of_Voters_TWO_or_MORE_RACES',
    'race_percent_UNDESIGNATED RACE': 'Percent_of_Voters_UNDESIGNATED_RACE',
    'race_percent_WHITE': 'Percent_of_Voters_WHITE',
    'percent_Other Race': 'Percent_of_Voters_Other_Race',
    'n_ASIAN': 'Number_of_Voters_ASIAN',
    'n_BLACK or AFRICAN AMERICAN': 'Number_of_Voters_BLACK_or_AFRICAN_AMERICAN',
    'n_INDIAN AMERICAN or ALASKA NATIVE': 'Number_of_Voters_INDIAN_AMERICAN_or_ALASKA_NATIVE',
    'n_OTHER': 'Number_of_Voters_OTHER',
    'n_TWO or MORE RACES': 'Number_of_Voters_TWO_or_MORE_RACES',
    'n_UNDESIGNATED RACE': 'Number_of_Voters_UNDESIGNATED_RACE',
    'n_WHITE': 'Number_of_Voters_WHITE',
    'n_Other Race': 'Number_of_Voters_Other_Race',
    'race_percent_NATIVE HAWAIIAN or PACIFIC ISLANDER': 'Percent_of_Voters_NATIVE_HAWAIIAN_or_PACIFIC_ISLANDER',
    'n_NATIVE HAWAIIAN or PACIFIC ISLANDER': 'Number_of_Voters_NATIVE_HAWAIIAN_or_PACIFIC_ISLANDER',
    'Latitude': 'Latitude',
    'Longitude': 'Longitude',
    'Saturday Count': 'Number_of_Saturdays_Open',
    'Sunday Count': 'Number_of_Sundays_Open',
    'Weekday Count': 'Number_of_Weekdays_Open',
    'Weekend Count': 'Number_of_Weekend_Days_Open',
    'Total Days Count': 'Total_Days_Open',
    'Total Hours Open': 'Total_Hours_Open',
    'Saturday': 'Saturday_Time_Ranges',
    'Sunday': 'Sunday_Time_Ranges',
    'Weekday': 'Weekday_Time_Ranges',
    'election_date': 'Election_Date',
}

In [1261]:
final_df_with_times.rename(columns=rename_dict, inplace=True)

In [1262]:
reordered_df = ['County', 'Election', 'Site_Name','Discrepancies', 'Address', 'CBOE_or_Alternate',
       'Site_Total', 'Number_of_SDR_Voters', 'Percent_SDR_Voters',
       'Percent_of_Voters_Age_18-25', 'Percent_of_Voters_Age_26-40',
       'Percent_of_Voters_Age_41-65', 'Percent_of_Voters_Age_Over_66',
       'Percent_of_Voters_Age_NA', 'Number_of_Voters_Age_18-25',
       'Number_of_Voters_Age_26-40', 'Number_of_Voters_Age_41-65',
       'Number_of_Voters_Age_Over_66', 'Number_of_Voters_Age_NA',
       'Percent_of_Voters_HISPANIC_or_LATINO',
       'Percent_of_Voters_NOT_HISPANIC_or_NOT_LATINO',
       'Percent_of_Voters_UNDESIGNATED_ETHNICITY',
       'Number_of_Voters_HISPANIC_or_LATINO',
       'Number_of_Voters_NOT_HISPANIC_or_NOT_LATINO',
       'Number_of_Voters_UNDESIGNATED_ETHNICITY', 'Percent_of_Voters_ASIAN',
       'Percent_of_Voters_BLACK_or_AFRICAN_AMERICAN',
       'Percent_of_Voters_INDIAN_AMERICAN_or_ALASKA_NATIVE',
       'Percent_of_Voters_OTHER', 'Percent_of_Voters_TWO_or_MORE_RACES',
       'Percent_of_Voters_UNDESIGNATED_RACE', 'Percent_of_Voters_WHITE','Percent_of_Voters_NATIVE_HAWAIIAN_or_PACIFIC_ISLANDER',
       'Percent_of_Voters_Other_Race',
       'Number_of_Voters_ASIAN', 'Number_of_Voters_BLACK_or_AFRICAN_AMERICAN',
       'Number_of_Voters_INDIAN_AMERICAN_or_ALASKA_NATIVE',
       'Number_of_Voters_OTHER', 'Number_of_Voters_TWO_or_MORE_RACES',
       'Number_of_Voters_UNDESIGNATED_RACE', 'Number_of_Voters_WHITE', 'Number_of_Voters_NATIVE_HAWAIIAN_or_PACIFIC_ISLANDER', 
       'Number_of_Voters_Other_Race', 
       'Latitude',
       'Longitude', 'Number_of_Saturdays_Open',
       'Number_of_Sundays_Open', 'Number_of_Weekdays_Open',
       'Number_of_Weekend_Days_Open', 'Total_Days_Open', 'Total_Hours_Open',
       'Saturday_Time_Ranges', 'Sunday_Time_Ranges', 'Weekday_Time_Ranges',
       'Election_Date']

In [1263]:
final_df_with_times = final_df_with_times[reordered_df]

In [1264]:
final_output_file = 'site_stats_' + datetime.today().strftime('%Y-%m-%d') + '_discrepancies.csv'
final_output_file

'site_stats_2024-04-22_discrepancies.csv'

In [1217]:
final_df_with_times.to_csv(final_output_file, index=False)

In [770]:
final_df_with_times.columns

Index(['County', 'Election', 'Site_Name', 'Address', 'CBOE_or_Alternate',
       'Site_Total', 'Number_of_SDR_Voters', 'Percent_SDR_Voters',
       'Percent_of_Voters_Age_18-25', 'Percent_of_Voters_Age_26-40',
       'Percent_of_Voters_Age_41-65', 'Percent_of_Voters_Age_Over_66',
       'Percent_of_Voters_Age_NA', 'Number_of_Voters_Age_18-25',
       'Number_of_Voters_Age_26-40', 'Number_of_Voters_Age_41-65',
       'Number_of_Voters_Age_Over_66', 'Number_of_Voters_Age_NA',
       'Percent_of_Voters_HISPANIC_or_LATINO',
       'Percent_of_Voters_NOT_HISPANIC_or_NOT_LATINO',
       'Percent_of_Voters_UNDESIGNATED_ETHNICITY',
       'Number_of_Voters_HISPANIC_or_LATINO',
       'Number_of_Voters_NOT_HISPANIC_or_NOT_LATINO',
       'Number_of_Voters_UNDESIGNATED_ETHNICITY', 'Percent_of_Voters_ASIAN',
       'Percent_of_Voters_BLACK_or_AFRICAN_AMERICAN',
       'Percent_of_Voters_INDIAN_AMERICAN_or_ALASKA_NATIVE',
       'Percent_of_Voters_OTHER', 'Percent_of_Voters_TWO_or_MORE_RACES',
   

In [403]:
# ISOCHRONE
import requests
import geojson
import time

In [9]:
df_unique = final_df_with_times[['County', 'Address', 'Latitude', 'Longitude']].drop_duplicates()

In [535]:
df_unique.head(), df_unique.shape

NameError: name 'df_unique' is not defined

In [11]:
MAPBOX_API_TOKEN = 'pk.eyJ1Ijoia2xhdXNtYXlyIiwiYSI6ImNsM20xZWFxejAwNmkza29mcHptN243aTYifQ.lfLe-OcdZ89adHFcD7r9tQ'

BASE_URL = 'https://api.mapbox.com/isochrone/v1/mapbox/walking/'

In [12]:
def create_isochrone(lat, lon):
    params = {
        'contours_minutes': [30],
        'polygons': 'true',
        'access_token': MAPBOX_API_TOKEN,
    }
    response = requests.get(f'{BASE_URL}{lon},{lat}', params=params)
    return response.json()

In [14]:
isochrone_features = []

for index, row in df_unique.iterrows():
    print(row)
    time.sleep(0.1)
    isochrone = create_isochrone(row['Latitude'], row['Longitude'])
    if 'features' in isochrone:
        # Extract the first feature's geometry from the isochrone response
        isochrone_geometry = isochrone['features'][0]['geometry']
        isochrone_feature = geojson.Feature(geometry=isochrone_geometry)
        # Here, you can add properties based on the row's data if needed
        isochrone_feature['properties'] = {'County': row['County'], 'Address': row['Address']}
        isochrone_features.append(isochrone_feature)

output_geojson = geojson.FeatureCollection(isochrone_features)
output_file_path = '30_min_walking.geojson'

with open(output_file_path, 'w') as output_file:
    geojson.dump(output_geojson, output_file, indent=2)

print(f'Isochrones created and saved to {output_file_path}')

County                                                ALAMANCE
Address      ALAMANCE COUNTY BOARD OF ELECTIONS OFFICE, 115...
Latitude                                             36.067659
Longitude                                           -79.402746
Name: 0, dtype: object
County                                                ALAMANCE
Address      COURT SERVICE BUILDING, 201 W ELM ST, GRAHAM, ...
Latitude                                              36.06959
Longitude                                           -79.402183
Name: 3, dtype: object
County                                             ALAMANCE
Address      COURT SERVICES, 201 W ELM ST, GRAHAM, NC 27253
Latitude                                           36.06959
Longitude                                        -79.402183
Name: 4, dtype: object
County                                                ALAMANCE
Address      ELMIRA COMMUNITY CENTER, 810 WICKER ST, BURLIN...
Latitude                                             36.10698

In [15]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from tqdm.notebook import tqdm

In [18]:
polygons = gpd.read_file('30_min_walking.geojson')  # or 'your_polygons.shp' for Shapefile
polygons['point_count'] = 0  # Initialize the count column

In [19]:
total_rows = sum(1 for row in open('all_snapshot_geocoded_nodups.csv', 'r'))  # Total number of rows in CSV
chunk_size = 100000  # Adjust based on your system's memory
total_chunks = (total_rows // chunk_size) + (1 if total_rows % chunk_size > 0 else 0)

In [20]:
with tqdm(total=total_chunks) as pbar:
    for chunk in pd.read_csv('all_snapshot_geocoded_nodups.csv', chunksize=chunk_size):
        # Convert each chunk of points to a GeoDataFrame
        points_gdf = gpd.GeoDataFrame(chunk, geometry=[Point(xy) for xy in zip(chunk.longitude, chunk.latitude)])
        
        # Spatial join points with polygons to count points in each polygon
        joined = gpd.sjoin(points_gdf, polygons, how='inner', op='within')
        
        # Update counts
        for index in joined['index_right'].unique():
            polygons.loc[index, 'point_count'] += joined[joined['index_right'] == index].shape[0]
        
        pbar.update(1)  # Update the progress bar

  0%|          | 0/74 [00:00<?, ?it/s]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/nq/tndcfcv94ld_dtqs_sgc70000000gq/T/ipykernel_47931/499721510.py:7: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined = gpd.sjoin(points_gdf, polygons, how='inner', op='within')
/var/folders/nq/tndcfcv94ld_dtqs_sgc70000000gq/T/ipykernel_47931/499721510.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('all_snapshot_geocoded_nodups.csv', chunksize=chunk_size):
/Library/Frameworks/Python.framework/Versions/3.11/lib/pyt

In [21]:
polygons.to_file('updated_30minwalking_polygons.geojson', driver='GeoJSON')  # or use '.shp' for Shapefile

In [ ]:
# Geocoding VR_Snapshots

In [22]:
import zipfile
import numpy as np
import os

In [23]:
columns_to_load = [
    'snapshot_dt',
    'county_id',
    'county_desc',
    'ncid',
    'status_cd',
    'voter_status_desc',
    'house_num',
    'half_code',
    'street_dir',
    'street_name',
    'street_type_cd',
    'street_sufx_cd',
    'unit_designator',
    'unit_num',
    'res_city_desc',
    'state_cd',
    'zip_code',
    'race_code',
    'ethnic_code',
    'party_cd',
    'sex_code',
    'age',
    'birth_place',
    'registr_dt',
    'age_group'
]

In [6]:
chunk_list = []
chunksize = 2 * 10 ** 6

In [7]:
zip_file_path = '/Users/klausmayr/Downloads/VR_Snapshot_20221206.zip'
file_name_inside_zip = 'VR_Snapshot_20221206.txt'

In [8]:
extraction_path = '/Users/klausmayr/Desktop/VR/ev_sites/vr_snapshots/'

In [9]:
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extract(file_name_inside_zip, extraction_path)

In [10]:
extracted_file_path = os.path.join(extraction_path, file_name_inside_zip)

In [11]:
for chunk in pd.read_csv(extracted_file_path, sep='\t', usecols=columns_to_load, chunksize=chunksize, encoding="utf-16"):
    chunk_list.append(chunk)

/var/folders/nq/tndcfcv94ld_dtqs_sgc70000000gq/T/ipykernel_75890/729970275.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(extracted_file_path, sep='\t', usecols=columns_to_load, chunksize=chunksize, encoding="utf-16"):
/var/folders/nq/tndcfcv94ld_dtqs_sgc70000000gq/T/ipykernel_75890/729970275.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(extracted_file_path, sep='\t', usecols=columns_to_load, chunksize=chunksize, encoding="utf-16"):
/var/folders/nq/tndcfcv94ld_dtqs_sgc70000000gq/T/ipykernel_75890/729970275.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(extracted_file_path, sep='\t', usecols=columns_to_load, chunksize=chunksize, encoding="utf-16"):
/var/folders/nq/tndcfcv94ld_dtqs_sgc70000000gq/T/ipykernel_75890/729970275.py:1: D

In [12]:
vr_snapshots_selects = pd.concat(chunk_list, ignore_index=True)

In [13]:
vr_snapshots_selects = vr_snapshots_selects[~vr_snapshots_selects['voter_status_desc'].isin(['REMOVED', 'DENIED'])]

In [24]:
print(len(vr_snapshots_selects))

NameError: name 'vr_snapshots_selects' is not defined

In [14]:
# print(vr_snapshots_selects.head().transpose())

In [15]:
sboe_address_points = pd.read_csv("address/address_points_sboe.txt", sep='\t',
                         names=['index','county_id','county_desc', 'st_address', 'city', 'zip_code','house_num','half_code', 'street_dir',
                               'street_name','street_type_cd','street_sufx_cd', 'lat_feet', 'long_feet', 'latitude','longitude'])

/var/folders/nq/tndcfcv94ld_dtqs_sgc70000000gq/T/ipykernel_75890/1402736925.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  sboe_address_points = pd.read_csv("address/address_points_sboe.txt", sep='\t',


In [45]:
# merge_columns = ['county_desc', 'zip_code', 'house_num','street_name', 'street_type_cd']
merge_columns = ['zip_code', 'house_num','street_name', 'street_type_cd']

In [17]:
vr_snapshots_selects['house_num'] = vr_snapshots_selects['house_num'].astype(str)
sboe_address_points['house_num'] = sboe_address_points['house_num'].astype(str)
vr_snapshots_selects['zip_code'] = vr_snapshots_selects['zip_code'].astype(str)
sboe_address_points['zip_code'] = sboe_address_points['zip_code'].astype(str)

In [18]:
vr_snapshots_with_address_points = pd.merge(vr_snapshots_selects, sboe_address_points[['latitude', 'longitude'] + merge_columns], on=merge_columns, how='left')

In [19]:
# print(vr_snapshots_with_address_points.head().transpose())

In [20]:
successful_merges = vr_snapshots_with_address_points.dropna(subset=['latitude', 'longitude']).shape[0]

In [21]:
unsuccessful_merges = vr_snapshots_with_address_points[vr_snapshots_with_address_points['latitude'].isna() & vr_snapshots_with_address_points['longitude'].isna()].shape[0]

print(f"Number of successful merges: {successful_merges}")
print(f"Number of rows without a merge: {unsuccessful_merges}")

Number of successful merges: 7294175
Number of rows without a merge: 434874


In [30]:
unsuccessful_merges_df = vr_snapshots_with_address_points[vr_snapshots_with_address_points['latitude'].isna() & vr_snapshots_with_address_points['longitude'].isna()]

# Export these rows to a CSV file
# unsuccessful_merges_df.to_csv('unsuccessful_merges.csv', index=False)

print(f"Exported {len(unsuccessful_merges_df)} rows to 'unsuccessful_merges.csv'")

Exported 434874 rows to 'unsuccessful_merges.csv'


In [22]:
nconemap_address_points = pd.read_csv("nc_onemap_addresses.csv", sep=',')

/var/folders/nq/tndcfcv94ld_dtqs_sgc70000000gq/T/ipykernel_75890/1347259824.py:1: DtypeWarning: Columns (7,9,10,11,13,14,22,23,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  nconemap_address_points = pd.read_csv("nc_onemap_addresses.csv", sep=',')


In [23]:
nconemap_address_points = nconemap_address_points.rename(columns={'County': 'county_desc', 'Post_Code': 'zip_code', 'Add_Number': 'house_num', 'St_Name': 'street_name', 'St_PosTyp': 'street_type_cd', 'LSt_PreDir': 'street_dir', 'DDLong': 'longitude', 'DDLat': 'latitude'})

In [24]:
street_type_mapping = {
    'ROAD': 'RD', 'LANE': 'LN', 'TRAIL': 'TRL', 'ALLEY': 'ALY', 'DRIVE': 'DR',
    'WAY': 'WAY', 'VIEW': 'VW', 'CIRCLE': 'CIR', 'STREET': 'ST', 'AVENUE': 'AVE',
    'SQUARE': 'SQ', 'BOULEVARD': 'BLVD', 'LOOP': 'LOOP', 'MEADOWS': 'MDWS',
    'ACRES': 'AC', 'HOLLOW': 'HOLW', 'FORK': 'FRK', 'RIDGE': 'RDG', 'COVE': 'CV',
    'HILL': 'HL', 'PLACE': 'PL', 'ESTATE': 'EST', 'HILLS': 'HLS', 'KNOLL': 'KNL',
    'COURT': 'CT', 'SPRINGS': 'SPGS', 'ESTATES': 'ESTS', 'PARK': 'PARK',
    'HEIGHTS': 'HTS', 'BEND': 'BND', 'PATH': 'PATH', 'CORNER': 'COR', 'PARKWAY': 'PKWY',
    'HIGHWAY': 'HWY', 'RUN': 'RUN', 'TRACE': 'TRCE', 'VILLAGE': 'VLG', 'BYPASS': 'BYP',
    'TERRACE': 'TER', 'CROSSING': 'XING', 'CREEK': 'CRK', 'PLAZA': 'PLZ', 'BRANCH': 'BR',
    'CENTER': 'CTR', 'MOUNTAIN': 'MTN', 'VALLEY': 'VLY', 'GREEN': 'GRN', 'KNOLLS': 'KNLS',
    'PASS': 'PASS', 'ISLAND': 'IS', 'POINT': 'PT', 'FOREST': 'FRST', 'ROW': 'ROW',
    'FLAT': 'FLT', 'REST': 'RST', 'HAVEN': 'HVN', 'VISTA': 'VIS', 'BLUFF': 'BLF',
    'GROVE': 'GRV', 'FARM': 'FM', 'OVERLOOK': 'OLK', 'POINTE': 'PTE', 'CREST': 'CRST',
    'CONNECTOR': 'CON', 'LANDING': 'LNDG', 'SPUR': 'SPUR', 'CROSS': 'CRS', 'EXTENSION': 'EXT',
    'WYND': 'WYND', 'WALK': 'WALK', 'TURN': 'TURN', 'GLEN': 'GLN', 'OVAL': 'OVL',
    'SHORES': 'SHRS', 'BRIDGE': 'BRG', 'HARBOR': 'HBR', 'FALLS': 'FLS', 'LAKE': 'LK',
    'EXPRESSWAY': 'EXPY', 'GARDEN': 'GDN', 'SPRING': 'SPG', '(GAP)': 'GAP', 'FLAT': 'FLT',
    'REST': 'RST', 'GARDENS': 'GDNS', 'FIELDS': 'FLDS', 'ANNEX': 'ANX', 'BAY': 'BY',
    'MEADOW': 'MDW', 'ORCHARD': 'ORCH', 'MILL': 'ML', 'PIKE': 'PIKE', 'FLATS': 'FLTS',
    'EDGE': 'EDG', 'TURNPIKE': 'TPKE', 'CURVE': 'CURV', 'LODGE': 'LDG', 'COVES': 'CVS',
    'FORD': 'FRD', 'FREEWAY': 'FWY', 'CAUSEWAY': 'CSWY', 'MANOR': 'MNR', 'NECK': 'NCK',
    'CHASE': 'CHSE', 'REACH': 'RCH', 'STATION': 'STA', 'PASSAGE': 'PSG', 'MEWS': 'MEWS',
    'DALE': 'DL', 'COMMON': 'CMN', 'LEDGE': 'LDG', 'COMMONS': 'CMNS'
}

In [25]:
def rename_street_types(street_rename_df, column_name, mapping):
    street_rename_df[column_name] = street_rename_df[column_name].map(mapping).fillna(street_rename_df[column_name])
    return street_rename_df

In [26]:
# Applying the function to your DataFrame
nconemap_address_points = rename_street_types(nconemap_address_points, 'street_type_cd', street_type_mapping)

In [27]:
nconemap_address_points = nconemap_address_points.dropna(subset=['zip_code', 'house_num'], how='any')

In [38]:
nconemap_address_points['zip_code'] = nconemap_address_points['zip_code'].astype(str)
nconemap_address_points['house_num'] = nconemap_address_points['house_num'].astype(str)
unsuccessful_merges_df['zip_code'] = unsuccessful_merges_df['zip_code'].astype(str)
unsuccessful_merges_df['house_num'] = unsuccessful_merges_df['house_num'].astype(str)

/var/folders/nq/tndcfcv94ld_dtqs_sgc70000000gq/T/ipykernel_75890/514359129.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unsuccessful_merges_df['zip_code'] = nconemap_address_points['zip_code'].astype(str)
/var/folders/nq/tndcfcv94ld_dtqs_sgc70000000gq/T/ipykernel_75890/514359129.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unsuccessful_merges_df['house_num'] = nconemap_address_points['house_num'].astype(str)


In [46]:
# FOR SOME REASON, MERGING THESE DATAFRAMES IN PYTHON IS NOT WORKING. I EXPORTED THE "unsuccessful_merges_df" AND "nconemap_address_points"
# DATAFRAMES AND DID THE MATCH IN R, WHICH RETURNED MANY MORE MATCHES. THAN IT DID IN PYTHON. NOT SURE WHY. BUT WHILE I FIGURE THAT OUT, 
# I'M JUST LOADING THE R MERGED DATAFRAME HERE:

# nconemap_merges = pd.merge(unsuccessful_merges_df, nconemap_address_points[['latitude', 'longitude'] + merge_columns], on=merge_columns, how='left')
# successful_onemap_merges = nconemap_merges.dropna(subset=['latitude_y', 'longitude_y']).shape[0]

# unsuccessful_onemap_merges = nconemap_merges[nconemap_merges['latitude_y'].isna() & nconemap_merges['longitude_y'].isna()].shape[0]

# print(f"Number of successful merges: {successful_onemap_merges}")
# print(f"Number of rows without a merge: {unsuccessful_onemap_merges}")

In [56]:
merged_in_r = pd.read_csv("unsuccessful_merges_merged_in_r.csv", sep=',')

/var/folders/nq/tndcfcv94ld_dtqs_sgc70000000gq/T/ipykernel_75890/1134228400.py:1: DtypeWarning: Columns (7,11) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_in_r = pd.read_csv("unsuccessful_merges_merged_in_r.csv", sep=',')


In [57]:
successful_onemap_merges = merged_in_r.dropna(subset=['latitude.y', 'longitude.y']).shape[0]

unsuccessful_onemap_merges = merged_in_r[merged_in_r['latitude.y'].isna() & merged_in_r['longitude.y'].isna()].shape[0]

print(f"Number of successful merges: {successful_onemap_merges}")
print(f"Number of rows without a merge: {unsuccessful_onemap_merges}")

Number of successful merges: 1330242
Number of rows without a merge: 182263


In [59]:
successful_sboe_merges_df = vr_snapshots_with_address_points.dropna(subset=['latitude', 'longitude'])

In [81]:
successful_sboe_merges_df['latlon_source'] = 'address_points_sboe.txt'
merged_in_r['latlon_source'] = 'AddressNC-addresses-02-21-2024.zip (NC Onemap)'

/var/folders/nq/tndcfcv94ld_dtqs_sgc70000000gq/T/ipykernel_75890/775946757.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  successful_sboe_merges_df['latlon_source'] = 'address_points_sboe.txt'


In [82]:
successful_sboe_merges_df.head()

,snapshot_dt,county_id,county_desc,ncid,status_cd,voter_status_desc,house_num,half_code,street_dir,street_name,...,ethnic_code,party_cd,sex_code,age,birth_place,registr_dt,age_group,latitude,longitude,latlon_source
0,2022-12-06,1,ALAMANCE,AA6924,A,ACTIVE,1816,,,EDGEWOOD,...,NL,UNA,M,75,NC,1968-04-20,Age Over 66,36.041383,-79.377864,address_points_sboe.txt
1,2022-12-06,1,ALAMANCE,AA21151,A,ACTIVE,705,,,CROFTWOOD,...,NL,UNA,M,86,NC,1964-10-24,Age Over 66,36.099129,-79.525971,address_points_sboe.txt
2,2022-12-06,1,ALAMANCE,AA27357,A,ACTIVE,708,,,KECK,...,UN,REP,M,69,NC,1972-01-17,Age Over 66,36.106113,-79.355647,address_points_sboe.txt
3,2022-12-06,1,ALAMANCE,AA27361,A,ACTIVE,708,,,KECK,...,NL,REP,F,74,NC,1973-05-17,Age Over 66,36.106113,-79.355647,address_points_sboe.txt
4,2022-12-06,1,ALAMANCE,AA27366,A,ACTIVE,1975,,,HAW RIVER-HOPEDALE,...,NL,DEM,F,81,NC,1982-09-18,Age Over 66,36.105205,-79.377956,address_points_sboe.txt


In [83]:
merged_in_r.head()

,snapshot_dt,county_id,county_desc,ncid,status_cd,voter_status_desc,house_num,half_code,street_dir,street_name,...,sex_code,age,birth_place,registr_dt,age_group,latitude.x,longitude.x,latitude.y,longitude.y,latlon_source
0,2022-12-06,1,ALAMANCE,AA23132,A,ACTIVE,4551,NaN,S,NC HWY 49,...,M,69,NC,1972-01-15,Age Over 66,NaN,NaN,NaN,NaN,AddressNC-addresses-02-21-2024.zip (NC Onemap)
1,2022-12-06,1,ALAMANCE,AA82195,A,ACTIVE,1453,NaN,N,NC HWY 87,...,U,57,NaN,1992-07-22,Age 41 - 65,NaN,NaN,NaN,NaN,AddressNC-addresses-02-21-2024.zip (NC Onemap)
2,2022-12-06,1,ALAMANCE,AA113774,A,ACTIVE,2422,NaN,S,NC HWY 119,...,F,59,NC,1999-03-01,Age 41 - 65,NaN,NaN,NaN,NaN,AddressNC-addresses-02-21-2024.zip (NC Onemap)
3,2022-12-06,1,ALAMANCE,AA145829,A,ACTIVE,5614,NaN,N,NC HWY 62,...,M,34,NC,2006-02-06,Age 26 - 40,NaN,NaN,NaN,NaN,AddressNC-addresses-02-21-2024.zip (NC Onemap)
4,2022-12-06,1,ALAMANCE,AA23053,A,ACTIVE,3542,NaN,S,NC HWY 87,...,M,82,NC,2016-02-18,Age Over 66,NaN,NaN,NaN,NaN,AddressNC-addresses-02-21-2024.zip (NC Onemap)


In [84]:
successful_onemap_merges_df = merged_in_r.dropna(subset=['latitude.y', 'longitude.y'])

In [86]:
successful_onemap_merges_df = successful_onemap_merges_df.drop(['latitude.x',	'longitude.x'], axis=1)

In [85]:
successful_onemap_merges_df.head()

,snapshot_dt,county_id,county_desc,ncid,status_cd,voter_status_desc,house_num,half_code,street_dir,street_name,...,sex_code,age,birth_place,registr_dt,age_group,latitude.x,longitude.x,latitude.y,longitude.y,latlon_source
9,2022-12-06,1,ALAMANCE,AA187800,I,INACTIVE,406,NaN,NaN,TRAIL ONE,...,F,30,VA,2016-02-18,Age 26 - 40,NaN,NaN,36.073845,-79.464180,AddressNC-addresses-02-21-2024.zip (NC Onemap)
12,2022-12-06,1,ALAMANCE,AA214087,A,ACTIVE,923,NaN,N,MEBANE,...,M,70,NaN,2020-09-25,Age Over 66,NaN,NaN,36.092037,-79.423935,AddressNC-addresses-02-21-2024.zip (NC Onemap)
15,2022-12-06,1,ALAMANCE,BY351338,A,ACTIVE,1720,NaN,NaN,OLD ST MARKS CHURCH,...,M,53,NC,2014-02-05,Age 41 - 65,NaN,NaN,36.063694,-79.510277,AddressNC-addresses-02-21-2024.zip (NC Onemap)
16,2022-12-06,1,ALAMANCE,BY351338,A,ACTIVE,1720,NaN,NaN,OLD ST MARKS CHURCH,...,M,53,NC,2014-02-05,Age 41 - 65,NaN,NaN,36.063820,-79.511055,AddressNC-addresses-02-21-2024.zip (NC Onemap)
17,2022-12-06,1,ALAMANCE,BY351338,A,ACTIVE,1720,NaN,NaN,OLD ST MARKS CHURCH,...,M,53,NC,2014-02-05,Age 41 - 65,NaN,NaN,36.064152,-79.510056,AddressNC-addresses-02-21-2024.zip (NC Onemap)


In [87]:
successful_onemap_merges_df = successful_onemap_merges_df.rename(columns={'latitude.y': 'latitude', 'longitude.y': 'longitude'})

In [88]:
print(successful_onemap_merges_df.columns)

Index(['snapshot_dt', 'county_id', 'county_desc', 'ncid', 'status_cd',
       'voter_status_desc', 'house_num', 'half_code', 'street_dir',
       'street_name', 'street_type_cd', 'street_sufx_cd', 'unit_designator',
       'unit_num', 'res_city_desc', 'state_cd', 'zip_code', 'race_code',
       'ethnic_code', 'party_cd', 'sex_code', 'age', 'birth_place',
       'registr_dt', 'age_group', 'latitude', 'longitude', 'latlon_source'],
      dtype='object')


In [89]:
print(successful_sboe_merges_df.columns)

Index(['snapshot_dt', 'county_id', 'county_desc', 'ncid', 'status_cd',
       'voter_status_desc', 'house_num', 'half_code', 'street_dir',
       'street_name', 'street_type_cd', 'street_sufx_cd', 'unit_designator',
       'unit_num', 'res_city_desc', 'state_cd', 'zip_code', 'race_code',
       'ethnic_code', 'party_cd', 'sex_code', 'age', 'birth_place',
       'registr_dt', 'age_group', 'latitude', 'longitude', 'latlon_source'],
      dtype='object')


In [90]:
all_successful_merges_df = pd.concat([successful_sboe_merges_df, successful_onemap_merges_df], ignore_index=True)

In [91]:
successful_merges = all_successful_merges_df.dropna(subset=['latitude', 'longitude']).shape[0]

unsuccessful_merges = all_successful_merges_df[all_successful_merges_df['latitude'].isna() & all_successful_merges_df['longitude'].isna()].shape[0]

print(f"Number of successful merges: {successful_merges}")
print(f"Number of rows without a merge: {unsuccessful_merges}")

Number of successful merges: 8624417
Number of rows without a merge: 0


In [92]:
all_successful_merges_df = all_successful_merges_df.drop_duplicates()

In [93]:
all_successful_merges_df.shape[0]

8122405

In [94]:
all_successful_merges_df.to_csv('all_successful_merges.csv', index=False)

In [95]:
merged_in_r = merged_in_r.drop(['latitude.x',	'longitude.x'], axis=1)

In [96]:
merged_in_r = merged_in_r.rename(columns={'latitude.y': 'latitude', 'longitude.y': 'longitude'})

In [101]:
all_unsuccessful_merges_df = merged_in_r[merged_in_r['latitude'].isna() & merged_in_r['longitude'].isna()]

In [102]:
all_unsuccessful_merges_df.head()

,snapshot_dt,county_id,county_desc,ncid,status_cd,voter_status_desc,house_num,half_code,street_dir,street_name,...,ethnic_code,party_cd,sex_code,age,birth_place,registr_dt,age_group,latitude,longitude,latlon_source
0,2022-12-06,1,ALAMANCE,AA23132,A,ACTIVE,4551,NaN,S,NC HWY 49,...,NL,UNA,M,69,NC,1972-01-15,Age Over 66,NaN,NaN,AddressNC-addresses-02-21-2024.zip (NC Onemap)
1,2022-12-06,1,ALAMANCE,AA82195,A,ACTIVE,1453,NaN,N,NC HWY 87,...,UN,REP,U,57,NaN,1992-07-22,Age 41 - 65,NaN,NaN,AddressNC-addresses-02-21-2024.zip (NC Onemap)
2,2022-12-06,1,ALAMANCE,AA113774,A,ACTIVE,2422,NaN,S,NC HWY 119,...,NL,DEM,F,59,NC,1999-03-01,Age 41 - 65,NaN,NaN,AddressNC-addresses-02-21-2024.zip (NC Onemap)
3,2022-12-06,1,ALAMANCE,AA145829,A,ACTIVE,5614,NaN,N,NC HWY 62,...,NL,REP,M,34,NC,2006-02-06,Age 26 - 40,NaN,NaN,AddressNC-addresses-02-21-2024.zip (NC Onemap)
4,2022-12-06,1,ALAMANCE,AA23053,A,ACTIVE,3542,NaN,S,NC HWY 87,...,UN,UNA,M,82,NC,2016-02-18,Age Over 66,NaN,NaN,AddressNC-addresses-02-21-2024.zip (NC Onemap)


In [103]:
all_unsuccessful_merges_df.to_csv('all_unsuccessful_merges.csv', index=False)

In [3]:
csv_path = 'ev_site_raw_data.csv'
data_csv = pd.read_csv(csv_path)

# Extract unique dates from the 'election' field
unique_dates_csv = sorted(data_csv['election'].unique())

unique_dates_csv

['2016-11',
 '2017-10',
 '2018-05',
 '2018-11',
 '2019-09',
 '2019-10',
 '2019-11',
 '2020-03',
 '2020-06',
 '2020-11',
 '2021-10',
 '2021-11',
 '2022-05',
 '2022-07',
 '2022-11']

In [ ]:
# MERGING EXISTING VR SNAPSHOT FILES